**IMPORTING PACKAGES**

In [ ]:
!pip install pandas mlxtend

In [3]:
# Import libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

**LOADING DATA**

In [ ]:
file_name = 'Autopro.csv'

In [ ]:
df = pd.read_csv(file_name)


In [ ]:
df.head()

**DATA PREPARATION**

In [ ]:
# Filter data for successful transactions
transactions = df[df['STATUS'].isin(['Shipped', 'Resolved'])]


In [ ]:
print(transactions)

In [ ]:
# Remove trailing spaces from the PRODUCTLINE column
transactions['PRODUCTLINE'] = transactions['PRODUCTLINE'].str.rstrip()

# Print the updated DataFrame
print(transactions)


In [ ]:
# Drop null values from the DataFrame
transactions = transactions.dropna()

# Remove duplicate transactions
transactions = transactions.drop_duplicates()


In [ ]:
# Group the transactions by ORDERNUMBER and PRODUCTLINE and sum the QUANTITY column
transaction_basket = transactions.groupby(['ORDERNUMBER', 'PRODUCTLINE'])['QUANTITYORDERED'].sum().unstack().reset_index().fillna(0).set_index('ORDERNUMBER')

# Convert the transaction basket to a boolean array
transaction_basket = transaction_basket.astype(bool)
transaction_basket = transaction_basket.astype(bool)

# Print the transaction basket
print(transaction_basket)


In [26]:
# Encode the transaction basket using one-hot encoding
encoded_vals = []
for col in transaction_basket.columns:
  encoded_vals.append(transaction_basket[col].astype('int'))

# Create a new DataFrame with the encoded values
encoded_df = pd.DataFrame(encoded_vals).T
encoded_df.columns = transaction_basket.columns


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# prompt: view the results of the new dataframe

# Print the encoded DataFrame
print(encoded_df)


**TRAINING MODEL**

In [ ]:
# Generate frequent itemsets with a minimum support of 0.05
frequent_itemsets = apriori(encoded_df, min_support=0.05, use_colnames=True)

# Print the frequent itemsets
print(frequent_itemsets)


In [ ]:
# Generate association rules with a minimum confidence of 0.5 and lift of 1.5
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# Print the association rules
print(rules)

**RECOMMENDATION**

In [27]:
# @title Default title text
# Sort the rules by lift in descending order
sorted_rules = rules.sort_values('lift', ascending=False)

# Print the top 10 rules with the highest lift
print(sorted_rules.head(10))

# Get the antecedents and consequents of the top 10 rules
antecedents = [list(antecedent)[0] for antecedent in sorted_rules['antecedents'].tolist()]
consequents = [list(consequent)[0] for consequent in sorted_rules['consequents'].tolist()]

# Create a list of recommendations
recommendations = []
for i in range(len(antecedents)):
  antecedent = antecedents[i]
  consequent = consequents[i]
  recommendation = f"If a customer buys {antecedent}, they are likely to also buy {consequent}."
  recommendations.append(recommendation)

# Print the recommendations
for recommendation in recommendations:
  print(recommendation)

                         antecedents         consequents  antecedent support  \
9        (Vintage Cars, Motorcycles)            (Planes)            0.065217   
13            (Vintage Cars, Trains)             (Ships)            0.086957   
7             (Classic Cars, Trains)  (Trucks and Buses)            0.105072   
11            (Vintage Cars, Planes)             (Ships)            0.155797   
14                   (Ships, Trains)      (Vintage Cars)            0.076087   
6         (Trucks and Buses, Trains)      (Classic Cars)            0.072464   
1                 (Trucks and Buses)      (Classic Cars)            0.228261   
12                   (Ships, Planes)      (Vintage Cars)            0.090580   
3                            (Ships)      (Vintage Cars)            0.210145   
8   (Vintage Cars, Trucks and Buses)      (Classic Cars)            0.119565   

    consequent support   support  confidence      lift  leverage  conviction  \
9             0.195652  0.054348    0.8

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
